In [3]:
!pip install iteround==1.0.2
!pip install pairing==0.1.3
!pip install scikit-multilearn==0.2.0
!pip install arff==0.9
!pip install category_encoders==2.1.0

  Created wheel for pairing: filename=pairing-0.1.3-cp36-none-any.whl size=2135 sha256=05e8f50ab4ddada0e3962238d271f0bc38df68f2572ca453913b00456a1bed10
  Stored in directory: /root/.cache/pip/wheels/a2/d8/9b/b97f0f0dac35d102b3671a0e0418f4d47493c0a97118172fc0
Successfully built pairing
     |████████████████████████████████| 92kB 3.4MB/s 
  Created wheel for arff: filename=arff-0.9-cp36-none-any.whl size=4970 sha256=1a7eeb974fe995f8300ef1bcd2037d051c7dacc5ca28e8335b9646a81caf5345
  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b
Successfully built arff
     |████████████████████████████████| 102kB 2.9MB/s 


In [0]:
## Basics
import os
import random
import pickle
import iteround
import itertools
import numpy as np
from scipy import special
from sklearn.cluster import KMeans, MiniBatchKMeans

class Hasher():
    random.seed(0)
    np.random.seed(0)
    os.environ['PYTHONHASHSEED']=str(0)
    """
    comb_algo : "rec", "prod"
    
    cluster_algo: "kmean", "mbkmean"
    """
    def __init__(self, comb_algo = "rec", cluster_algo= "kmean"):
        self.comb_algo = comb_algo
        self.cluster_algo = cluster_algo

    def combinations_prod(self, n, tot):
        #### src: https://codereview.stackexchange.com/questions/190122/permutations-with-a-sum-constraint
        def combinations_prod_inner(n, tot):
            items = list(range(0,tot+1,1))
            combinations = pd.DataFrame(list(filter(lambda x: np.sum(x)==tot, 
                                                  list(itertools.product(items, repeat=n)))))
            return(combinations.as_matrix())

        res = combinations_prod_inner(n, tot)
        res = np.array(res)
        res = res/res.sum(axis=1)[:, np.newaxis]
        return res


    def combinations_recursive(self, n, tot):
        #### src: https://codereview.stackexchange.com/questions/190122/permutations-with-a-sum-constraint
        def combinations_recursive_inner(n, buf, gaps, tsum, accum, tot):
            if gaps == 0:
                accum.append(list(buf))
            else:
                for x in range(0, tot+1):
                    if tsum + x + (gaps - 1) * tot < tot:
                        continue
                    if tsum + x > tot:
                        break
                    combinations_recursive_inner(n, buf + [x], gaps - 1, tsum + x, accum, tot)
        
        res = []
        combinations_recursive_inner(n, [], n, 0, res, tot)
        res = np.array(res)
        res = res/res.sum(axis=1)[:, np.newaxis]
        return res

    
    
    def build_hasher(self, context_size, bin_size, dec_digits=1, saving=True):
        ### src:https://en.wikipedia.org/wiki/Stars_and_bars_(combinatorics)
        num_tot_hsits = special.comb(((10**dec_digits)+context_size-1), context_size-1)
        print("Total number of possible contexts(histograms):", num_tot_hsits)
        if self.comb_algo == "rec":
            all_hists = self.combinations_recursive(context_size, 10**dec_digits)
        elif self.comb_algo == "prod":
            all_hists = self.combinations_prod(context_size, 10**dec_digits)
        
        #### To clustering histograms
        print("Clustering...")
        if self.cluster_algo == "kmean":
            kmeans = KMeans(n_clusters=2**bin_size,
                            n_jobs = -1,
                            random_state=0).fit(all_hists)
        elif self.cluster_algo == "mbkmean":
            kmeans = MiniBatchKMeans(n_clusters=2**bin_size, 
                                     batch_size = bin_size*bin_size,
                                     init_size=2**bin_size,
                                     n_init=bin_size,
                                     random_state=0).fit(all_hists)
        
        #### To order clustering labels from highest to lowest
        idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
        re_indexer = np.zeros_like(idx)
        re_indexer[idx] = np.arange(2**bin_size)
        
        if saving:
            print("Saving...")
            save_dir = "encoders_repo"
            f_name =  "hasher_"+str(context_size)+"_"+str(dec_digits)+"_"+str(bin_size)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            with open(save_dir+"/kmeans_"+f_name+".pkl", 'wb') as fid:
                pickle.dump(kmeans, fid)
            np.save(save_dir+"/re_indexer_"+f_name+".npy", re_indexer)
        
        print("Completed!")
        return kmeans, re_indexer
    
    def get_hasher(self, context_size, bin_size, dec_digits=1):
        save_dir = "encoders_repo"
        f_name =  "hasher_"+str(context_size)+"_"+str(dec_digits)+"_"+str(bin_size)
        with open(save_dir+"/kmeans_"+f_name+".pkl", 'rb') as fid:
            kmeans = pickle.load(fid)
        re_indexer = np.load(save_dir+"/re_indexer_"+f_name+".npy")
        
        return kmeans, re_indexer

## Building an Encoder
To build your own encoder, you need to set the following three values:
1. `context_size` which is called `d` in the paper.
2. `bin_size` which is `log_2 (k)` in the paper. (in the paper we use three different values for `k`: 2^5,2^7, and 2^10)
3. `dec_digits` whic is called `q`.

You also can use either `kmean` or `mbkmean`. The former is much more accurate, however the latter is much more faster.
> more info: https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html

In [5]:
hasher = Hasher(comb_algo = "rec", cluster_algo= "kmean")
hasher.build_hasher(context_size=10, ## This is `d` in the paper
                    bin_size=5, ## This is `log_2(k)` in the paper
                    dec_digits=1, ## This is `q` in the paper
                    saving=True)

Total number of possible contexts(histograms): 92378.0
Clustering...
Saving...
Completed!


(KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=32, n_init=10, n_jobs=-1, precompute_distances='auto',
        random_state=0, tol=0.0001, verbose=0),
 array([21,  6, 14,  2, 12,  4,  5,  9,  8, 25, 22, 30, 31, 20,  7, 18,  3,
        17, 15, 13, 24, 28,  0, 29,  1, 10, 19, 11, 16, 27, 26, 23]))

In [6]:
hasher.get_hasher(context_size=10, bin_size=10,dec_digits=1)

(KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=1024, n_init=10, n_jobs=-1, precompute_distances='auto',
        random_state=0, tol=0.0001, verbose=0),
 array([248,  67, 413, ..., 369, 726, 528]))

### What is the Next Step?

Build as much as **Encoder** you need for your desired `context_size`, `bin_size`, and `dec_digits`. Then you can run one of the following notebooks:
* 2_synthetic_exp.ipynb
* 2_mlc_exp.ipynb
* 2_criteo_exp.ipynb